# Yolov8 Model Training on nuImages


### Installing the Necessary Dependencies

In [ ]:
!pip install clearml

In [ ]:
!pip install ultralytics

In [ ]:
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY= # Insert your key
%env CLEARML_API_SECRET_KEY= # Insert your key

In [ ]:
!wandb login #Insert key

## Downlaoding and Extracting the Dataset.


### Download via Google Cloud

In [ ]:
#If this is being run in Kaggle, make sure that these files are not stored in the working dircetory
%cd ../../
%pwd

In [ ]:
!wget https://storage.googleapis.com/nuimages/nuImages-train-1.tar.gz
!wget https://storage.googleapis.com/nuimages/nuImages-train-2.tar.gz
!wget https://storage.googleapis.com/nuimages/nuImages-train-3.tar.gz
!wget https://storage.googleapis.com/nuimages/nuImages-train-4.tar.gz
!wget https://storage.googleapis.com/nuimages/nuImages-train-5.tar.gz
!wget https://storage.googleapis.com/nuimages/nuImages-train-6.tar.gz
!wget https://storage.googleapis.com/nuimages/nuImages-train-7.tar.gz
!wget https://storage.googleapis.com/nuimages/nuImages-train-8.tar.gz
!wget https://storage.googleapis.com/nuimages/nuImages-train-9.tar.gz
!wget https://storage.googleapis.com/nuimages/nuImages-val-1.tar.gz
!wget https://storage.googleapis.com/nuimages/nuImages-val-2.tar.gz
!wget https://storage.googleapis.com/nuimages/nuImages-yaml.tar.gz

In [ ]:
!wget https://storage.googleapis.com/nuimages/best.pt
!wget https://storage.googleapis.com/nuimages/last.pt

### Archive Extraction

In [ ]:
!tar -xf nuImages-train-1.tar.gz -C ./
!tar -xf nuImages-train-2.tar.gz -C ./
!tar -xf nuImages-train-3.tar.gz -C ./
!tar -xf nuImages-train-4.tar.gz -C ./
!tar -xf nuImages-train-5.tar.gz -C ./
!tar -xf nuImages-train-6.tar.gz -C ./
!tar -xf nuImages-train-7.tar.gz -C ./
!tar -xf nuImages-train-8.tar.gz -C ./
!tar -xf nuImages-train-9.tar.gz -C ./
!tar -xf nuImages-val-1.tar.gz -C ./
!tar -xf nuImages-val-2.tar.gz -C ./
!tar -xf nuImages-yaml.tar.gz -C ./

Optionally, you can delete the archives that were unpacked to free up space.

In [ ]:
%rm nuImages-train-1.tar.gz
%rm nuImages-train-2.tar.gz
%rm nuImages-train-3.tar.gz
%rm nuImages-train-4.tar.gz
%rm nuImages-train-5.tar.gz
%rm nuImages-train-6.tar.gz
%rm nuImages-train-7.tar.gz
%rm nuImages-train-8.tar.gz
%rm nuImages-train-9.tar.gz
%rm nuImages-val-1.tar.gz
%rm nuImages-val-2.tar.gz
%rm nuImages-yaml.tar.gz

## Training

In [ ]:
from ultralytics import YOLO
from clearml import Task

In [ ]:
ckpt = 'last.pt'  #weights of the model
yaml_path = '/datasets/nuImages/data.yaml' #path to the dataset yaml file
project_name = 'nuimages-yolo' # for clearML
train_name = 'train' # for clearML


model = YOLO(ckpt)
resddults = model.train(data=yaml_path, project=project_name, name=train_name,
                        imgsz=960, epochs=4, warmup_epochs=0,
                        optimizer='AdamW', cos_lr=False, lr0=50*1e-6,
                        save=True, device=[0,1])

In [ ]:
#If in Kaggle, run this to obtain the results in the output folder
%cp -r /nuimages-yolo/ /kaggle/working/nuimages-yolo